In [11]:
homeUrl = "http://www.controller.com/"
siteUrl = homeUrl + "listings/aircraft/for-sale/list/"
print(siteUrl)

proxies = {'http': 'http://<<USERID>>:<<PASSWORD>>@23.19.51.245:40791/'}

http://www.controller.com/listings/aircraft/for-sale/list/


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import zipfile

manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
          singleProxy: {
            scheme: "http",
            host: "23.19.51.245",
            port: parseInt(40791)
          },
          bypassList: ["foobar.com"]
        }
      };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "<<USERID>>",
            password: "<<PASSWORD>>"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
"""


pluginfile = 'proxy_auth_plugin.zip'

with zipfile.ZipFile(pluginfile, 'w') as zp:
    zp.writestr("manifest.json", manifest_json)
    zp.writestr("background.js", background_js)

co = Options()
co.add_argument("--start-maximized")
co.add_argument('--dns-prefetch-disable')
#co.add_argument('--disable-extensions')
co.add_extension(pluginfile)

In [7]:
driver = webdriver.Chrome(chrome_options=co)

In [12]:
import time
from bs4 import BeautifulSoup

# set timeout information
driver.set_page_load_timeout(50)

finished = 0
fails = 0
while finished == 0 and fails < 8:
    try:
        driver.get(homeUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        fails = fails + 1
        time.sleep(5)

In [14]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<input type="submit" value="Search" alt="Search Aircraft For Sale at Controller.com." class="span12  m-bottom-10">
    buttonCss = 'input[alt="Search Aircraft For Sale at Controller.com."]'
    
    # Wait for button to show up
    buttonTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, buttonCss))
    )
    
    buttonTag.click()
except Exception as e:
    print(str(e))

In [15]:
# Wait for button to disappear
WebDriverWait(driver, 30).until_not(
    EC.visibility_of_element_located((By.CSS_SELECTOR, buttonCss))
)

siteSoup = BeautifulSoup(driver.page_source, 'lxml')

In [16]:
#<span id="listings-total-pages"> 265</span>
siteLinks = siteSoup.select('#listings-total-pages')
#print(siteLinks)

# Get the max page link
max_page = 0
if len(siteLinks):
    max_page = int(siteLinks[0].get_text().strip())
print(max_page)

[<span id="listings-total-pages"> 265</span>]
265


In [20]:
from random import randint

i = randint(1, max_page)
tagUrl = "?".join([siteUrl, "page="+str(i)])
print(tagUrl)

finished = 0
fails = 0
while finished == 0 and fails < 8:
    try:
        driver.get(tagUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        fails = fails + 1
        time.sleep(5)
tagSoup = BeautifulSoup(driver.page_source, 'lxml')

# Get the title of the page to prove we are progressing
tagTitle = tagSoup.select("head > title")
titleString = tagTitle[0].get_text().strip()
print(titleString)

http://www.controller.com/listings/aircraft/for-sale/list/?page=191
Controller.com | FOKKER 50 For Sale - 6571 Listings - Page 191.


In [24]:
#<div>Phone:   (225) 454-2117</div>
infoSoups = tagSoup.select('div.col.dealer-info.cf')

infoSoup = infoSoups[randint(0,len(infoSoups)-1)]

In [31]:
import re

phoneRe = re.compile(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}")
phoneTag = infoSoup.select('div:nth-of-type(3)')
match = phoneRe.search(phoneTag[0].get_text().strip())
if match:
    phone = match.group()
else:
    phone = ""
print(phone)

if len(phone):
    
    itemSoup = infoSoup.parent
    #print(itemSoup)

    #<span class="">Aviation Advisors Intl</span>
    seller = infoSoup.select('div:nth-of-type(1) > span')
    if(len(seller)):
        seller = seller[0].get_text().strip()
        seller = re.sub(r'\s+', ' ', seller)
    else:
        seller = "Unknown"
    print(seller)
    
    #<a href="/listings/aircraft/for-sale/1453947/2008-learjet-60xr">2008 LEARJET 60XR</a>
    model = itemSoup.select('div.listing-name > a')
    if(len(model)):
        model = model[0].get_text().strip()
        model = re.sub(r'\s+', ' ', model)
    else:
        model = "Unknown"
    print(model)
    
    #<span class="nobr">USD $3,495,000</span>
    price = itemSoup.select('div.price.column7.right.fluid-mobile > div > span.nobr')
    if(len(price)):
        price = price[0].get_text().strip()
    else:
        price = "Unknown"
    print(price)

(907) 486-5155
Kingfisher Air Inc
2000 FOUND AIRCRAFT CO BUSH HAWK XP
USD $229,000


In [32]:
import re

phoneRe = re.compile(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}")
for i in range(1, max_page+1):
    tagUrl = "?".join([siteUrl, "page="+str(i)])

    finished = 0
    fails = 0
    while finished == 0 and fails < 8:
        try:
            driver.get(tagUrl)
            finished = 1
        except Exception as e:
            print(str(e))
            fails = fails + 1
            time.sleep(5)
    tagSoup = BeautifulSoup(driver.page_source, 'lxml')
    
    infoSoups = tagSoup.select('div.col.dealer-info.cf')
    for infoSoup in infoSoups:
        
        phoneTag = infoSoup.select('div:nth-of-type(3)')
        match = phoneRe.search(phoneTag[0].get_text().strip())
        if match:
            phone = match.group()
        else:
            phone = ""
        if len(phone):

            itemSoup = infoSoup.parent
            
            seller = infoSoup.select('div:nth-of-type(1) > span')
            if(len(seller)):
                seller = seller[0].get_text().strip()
                seller = re.sub(r'\s+', ' ', seller)
            else:
                seller = "Unknown"

            model = itemSoup.select('div.listing-name > a')
            if(len(model)):
                model = model[0].get_text().strip()
                model = re.sub(r'\s+', ' ', model)
            else:
                model = "Unknown"

            price = itemSoup.select('div.price.column7.right.fluid-mobile > div > span.nobr')
            if(len(price)):
                price = price[0].get_text().strip()
            else:
                price = "Unknown"
            
            bigFile = open('controller.txt', 'a', encoding='utf-8')
            bigFile.write(seller + ': ' + model + '\t' + price + '\t' + phone + '\n')
            bigFile.close()

In [33]:
driver.quit()